From https://docs.dgl.ai/en/0.6.x/tutorials/blitz/4_link_predict.html

In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from dgl.data import DGLDataset
import numpy as np
import scipy.sparse as sp
import networkx as nx

Using backend: pytorch


In [2]:
edges = pd.read_csv('../data/twitch_gamers/large_twitch_edges.csv', header=0)
edges.columns = ['source', 'target']
features = pd.read_csv('../data/twitch_gamers/large_twitch_features.csv', header=0)

In [4]:
G = nx.from_pandas_edgelist(edges, source='source', target='target', create_using=nx.Graph())
degree_dict = dict(G.degree)
selected_nodes = []
for i in degree_dict:
    if degree_dict[i] >= 500:
        selected_nodes.append(i)
        
edges_filtered = edges[edges["source"].isin(selected_nodes)]
edges_filtered = edges[edges["target"].isin(selected_nodes)]

In [5]:
features['created_at'] = pd.to_datetime(features['created_at'])
features['updated_at'] = pd.to_datetime(features['updated_at'])

features['delta_days'] = (features['updated_at'] - features['created_at']).dt.total_seconds()/(60*60*24)

features['language'] = pd.factorize(features['language'])[0]

In [6]:
nodes_filtered = set(edges_filtered['source'].tolist() + edges_filtered['target'].tolist())
features_filtered = features.loc[nodes_filtered]

In [7]:
Sdict = dict(zip(nodes_filtered, range(len(nodes_filtered))))
edges_filtered['source'] = edges_filtered['source'].map(Sdict)
edges_filtered['target'] = edges_filtered['target'].map(Sdict)

C:\Users\Peter\AppData\Local\Temp/ipykernel_10508/345259505.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edges_filtered['source'] = edges_filtered['source'].map(Sdict)
C:\Users\Peter\AppData\Local\Temp/ipykernel_10508/345259505.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edges_filtered['target'] = edges_filtered['target'].map(Sdict)


In [8]:
class TwitchGamersDataset(DGLDataset):
    def __init__(self):
        super().__init__(name='twitch_gamers')

    def process(self):
        nodes_data = features_filtered
        edges_data = edges_filtered

        node_features = torch.from_numpy(nodes_data[['views', 'mature', 'life_time', 'delta_days', 'dead_account', 'language']].to_numpy()).float()
        node_labels = torch.from_numpy(nodes_data['affiliate'].astype('category').cat.codes.copy().to_numpy())
        edges_src = torch.from_numpy(edges_data['source'].to_numpy())
        edges_dst = torch.from_numpy(edges_data['target'].to_numpy())

        self.graph = dgl.graph((edges_src, edges_dst), num_nodes=nodes_data.shape[0])
        self.graph.ndata['feat'] = node_features
        self.graph.ndata['label'] = node_labels

        # If your dataset is a node classification dataset, you will need to assign
        # masks indicating whether a node belongs to training, validation, and test set.
        n_nodes = nodes_data.shape[0]
        n_train = int(n_nodes * 0.6)
        n_val = int(n_nodes * 0.2)
        train_mask = torch.zeros(n_nodes, dtype=torch.bool)
        val_mask = torch.zeros(n_nodes, dtype=torch.bool)
        test_mask = torch.zeros(n_nodes, dtype=torch.bool)
        train_mask[:n_train] = True
        val_mask[n_train:n_train + n_val] = True
        test_mask[n_train + n_val:] = True
        self.graph.ndata['train_mask'] = train_mask
        self.graph.ndata['val_mask'] = val_mask
        self.graph.ndata['test_mask'] = test_mask

    def __getitem__(self, i):
        return self.graph

    def __len__(self):
        return 1

dataset = TwitchGamersDataset()
graph = dataset[0]

print(graph)

Graph(num_nodes=37281, num_edges=567609,
      ndata_schemes={'feat': Scheme(shape=(6,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int8), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})


In [9]:
# Split edge set for training and testing
u, v = graph.edges()

eids = np.arange(graph.number_of_edges())
eids = np.random.permutation(eids)
test_size = int(len(eids) * 0.1)
train_size = graph.number_of_edges() - test_size
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

# Find all negative edges and split them for training and testing
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
adj_neg = 1 - adj.todense() - np.eye(graph.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)

neg_eids = np.random.choice(len(neg_u), graph.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]

ValueError: operands could not be broadcast together with shapes (37281,37277) (37281,37281) 